In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, count, date_format, to_timestamp, avg, stddev, udf, pandas_udf, array_contains, lit, when
from pyspark.sql.types import StructType, StructField, StringType, ArrayType
import pyspark.pandas as ps
import nltk
import pyspark
from nltk.tokenize import word_tokenize
from nltk.corpus import words
import pandas as pd
import pyarrow
import sys
import os
from pyspark.sql.window import Window


In [2]:
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

In [3]:
# Initialize Spark session

# SparkSession.getActiveSession().stop()

spark = SparkSession.builder \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
        .appName("TweetDistribution").getOrCreate()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

file_path = "Datasets/archive/Bitcoin_tweets.csv"
# Load CSV (modify options if needed)
inputDF = spark.read.csv(
    file_path, 
    header=True,  # Use the first row as column names
    inferSchema=True,  # Infer data types
    multiLine=True,  # Handle newlines within fields
    escape='"',  # Escape character for double quotes
    quote='"',  # Define the quote character

    # Different depending on the document
    sep=",",  # Specify the correct delimiter


    mode="PERMISSIVE"  # Handle malformed rows gracefully
)
inputDF.printSchema()

root
 |-- user_name: string (nullable = true)
 |-- user_location: string (nullable = true)
 |-- user_description: string (nullable = true)
 |-- user_created: string (nullable = true)
 |-- user_followers: string (nullable = true)
 |-- user_friends: string (nullable = true)
 |-- user_favourites: string (nullable = true)
 |-- user_verified: string (nullable = true)
 |-- date: string (nullable = true)
 |-- text: string (nullable = true)
 |-- hashtags: string (nullable = true)
 |-- source: string (nullable = true)
 |-- is_retweet: boolean (nullable = true)



In [4]:
# Change timestamp to date only
date_checkDF = inputDF.withColumn("time", date_format(to_timestamp(col("date")), "HH")) \
    .withColumn("date", date_format(to_timestamp(col("date")), "yyyy-MM-dd")) \
    .filter(col("time").isNotNull()).select("date", "time", "text")

In [30]:
import pyspark.sql.functions as F

In [31]:
countDF = date_checkDF.groupBy("time", "date").agg(count("*").alias("tweets_per_hour"))
distributionDF = countDF.groupBy("time").agg(
    avg("tweets_per_hour").alias("avg"),
    stddev("tweets_per_hour").alias("stddev")
).cache()
total_count = distributionDF.agg(F.sum("avg").alias("total_count")).collect()[0]["total_count"]

ratioDF = distributionDF.withColumn("ratio", col("avg") / total_count)

In [36]:
# Convert Spark DataFrame to Pandas-on-Spark DataFrame
psdf = ratioDF.to_pandas_on_spark()

# Set "word" as the index and select the columns to plot
df = psdf.set_index("time")[["avg", "stddev"]].sort_index()
df.plot.bar()

c:\Users\pryty\anaconda3\envs\spark_env\lib\site-packages\pyspark\sql\dataframe.py:5725: FutureWarning:

DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'variable=avg<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'avg',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'avg',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
                          '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23'],
                         dtype=object),
              'xaxis': 'x',
              'y': {'bdata': ('9DzP8zwjikDyeDwej7eHQA55Q96Qj4' ... 'sQXreWkkAfVg+rh6mQQGUJbNpP3o5A'),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'variable=stddev<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'stddev',
              'marker': {'color': '#EF553B', 'pattern': {'shape': ''}},
              'name': 'stddev',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
                          '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23'],
                         dtype=object),
              'xaxis': 'x',
              'y': {'bdata': ('/pVVEbDVfkDnzQAB5rl8QH/nFEurf3' ... 'XYZYEqhkA1L/2liymDQNo20G+5b4FA'),
                    'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'time'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'value'}}}
})

In [37]:
df = psdf.set_index("time")[["ratio"]].sort_index()
df.plot.bar()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'variable=ratio<br>time=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'ratio',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': 'ratio',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
                          '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23'],
                         dtype=object),
              'xaxis': 'x',
              'y': {'bdata': ('JyECxJ3Hnz90FK4wLNacPxUSb+aLpZ' ... 'PqKPCZpj+9O3DDSkKkPzJPCJMOxKI/'),
                    'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'margin': {'t': 60},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'time'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'value'}}}
})

In [40]:
# Estimation regarding my scraping:
# I got 13k tweets in 40 min:
val = 13_000 * 60/40    # Aprox tweets between 00 and 01
val = val / 0.031       # Aprox tweets in a day
val

629032.2580645161

In [41]:
# If i want the tweets scraped to be around 1000:
ratio_requiered = 1000 / val
ratio_requiered

0.0015897435897435897